In [2]:
import ollama

model = "llama3.2:1b"
message_content = 'why do leaves change in the fall?'
message = {'role': 'user', 'content': message_content}

# Try to call ollama directly - this assumes Ollama is already installed, along with the target model
ollama.chat(model=model,messages=[message])

{'model': 'llama3.2:1b',
 'created_at': '2024-10-11T00:33:17.799145Z',
 'message': {'role': 'assistant',
  'content': "Leaves changing color in the fall is a natural process that occurs when the days get shorter and the weather gets cooler. This process is triggered by the decrease in daylight hours, typically around September or October in the Northern Hemisphere.\n\nHere's what happens:\n\n1. **Shorter days**: As the Earth continues its orbit around the Sun, the amount of sunlight that reaches the Earth's surface decreases. In the fall, the days get shorter, which means there are fewer hours of daylight for plants to photosynthesize.\n2. **Cooler temperatures**: As the weather cools down, the trees slow down their growth rate and prepare for winter. This is because they need to conserve energy and resources to survive the cold temperatures.\n3. **Increased sugar concentration**: In the fall, trees stop producing as much chlorophyll (the green pigment that helps plants absorb sunlight

In [3]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model=model)

In [74]:
from langchain_core.prompts import PromptTemplate

RAG_PROMPT_TEMPLATE = """\
<|start_header_id|>system<|end_header_id|>
You are a helpful potty training assistant. You answer user questions based on context. If you can't answer the question with the context, say you don't know.
Context:
{context}
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
User Question:
{query}
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

rag_prompt = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

In [75]:
rag_chain = rag_prompt | llm

In [76]:
rag_chain.invoke({"query" : "How old is Carl?", "context" : "Carl is a sweet dude, he's 40."})

'Carl is 40 years old.'

In [7]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

In [8]:
import os

files = os.listdir("data")
files = [os.path.join("data", file) for file in files]

print(files[0])

data/Planning for Care Costs _ Alzheimer's Association.html


In [60]:
from langchain_community.document_loaders import RecursiveUrlLoader

# This example uses `beautifulsoup4` and `lxml`
import re
from bs4 import BeautifulSoup

def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    
    # Remove unwanted tags
    for tag in soup(['nav', 'footer', 'header', 'aside', 'script', 'style']):
        tag.decompose()
    
    # Extract text
    text = soup.get_text(separator=' ', strip=True)
    
    # Clean up whitespace
    clean_text = re.sub(r'\s+', ' ', text).strip()
    
    return clean_text

loader = RecursiveUrlLoader("https://pottygenius.com/blogs/blog",
                            extractor=bs4_extractor)
docs = loader.load()

/var/folders/qk/rnr0wf6j3xj452rpryymzb200000gs/T/ipykernel_98857/796195323.py:8: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html, "lxml")
/opt/anaconda3/lib/python3.9/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSou

In [66]:
unwanted_terms = str.split("arrow-right cart chevron-down chevron-left chevron-right chevron-up close menu minus play plus search share user email pinterest facebook instagram snapchat tumblr twitter vimeo youtube subscribe dogecoin dwolla forbrugsforeningen litecoin amazon_payments american_express bitcoin cirrus discover fancy interac jcb master paypal stripe visa diners_club dankort maestro trash"," ")
unwanted_terms.extend(["tell your story", "free undies","shop free undies!"])
for doc in docs:
    content = doc.page_content.lower()
    for term in unwanted_terms:
        content = content.replace(term,"")
    doc.page_content=content

In [67]:
print(len(docs))
print(docs[0])


30
page_content='potty genius blog                                             potty genius blog  games  shop ! potty genius blog — potty training boys — potty training girls — potty training methods  —  stories games  shop get ready to train! get ready to train it all starts with changing the mindset. we can’t just tell children it's time to potty... by marshall mizrahi potty genius blog pull ups®? potty genius blog recognize your child's toilet training readiness potty genius blog eva shockey on potty training potty genius blog shannen michaela on elimination communication potty genius blog potty training a child with down syndrome potty genius blog potty training boys and girls potty training is challenging regardless of your toddler’s gender. that said, potty training boys is a bit different than potty training girls. while it is obvious that males and females use the bathroom differently, there are some other distinct potty training differences parents may run into when potty trai

In [69]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,       # Adjust based on your needs
    chunk_overlap=200,     # Overlap to maintain context
)
split_docs = text_splitter.split_documents(docs)

print(f"len(docs): {len(docs)}, len(split_docs):{len(split_docs)}")

len(docs): 30, len(split_docs):109


In [71]:
for i in range(4): print(split_docs[i])

page_content='potty genius blog                                             potty genius blog  games  shop ! potty genius blog — potty training boys — potty training girls — potty training methods  —  stories games  shop get ready to train! get ready to train it all starts with changing the mindset. we can’t just tell children it's time to potty... by marshall mizrahi potty genius blog pull ups®? potty genius blog recognize your child's toilet training readiness potty genius blog eva shockey on potty training potty genius blog shannen michaela on elimination communication potty genius blog potty training a child with down syndrome potty genius blog potty training boys and girls potty training is challenging regardless of your toddler’s gender. that said, potty training boys is a bit different than potty training girls. while it is obvious that males and females use the bathroom differently, there are some other distinct potty training differences parents may run into when potty trainin

In [72]:
from langchain_qdrant import QdrantVectorStore
url="http://localhost:6333"

qdrant_vector_store = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url=url,
    prefer_grpc=True,
    collection_name="PottyTraining",
)

In [73]:
retriever = qdrant_vector_store.as_retriever(
    search_type="mmr",  # Options: 'similarity', 'mmr', etc.
    search_kwargs={"k": 5}     # Number of documents to retrieve
)
retriever.invoke("How is potty training boys different from potty training girls")

[Document(metadata={'language': None, 'title': '\n  Potty Training Boys and Girls – Potty Genius\n  ', 'content_type': 'text/html; charset=utf-8', 'source': 'https://pottygenius.com/blogs/blog/potty-training-differences-in-boys-and-girls', '_id': '88a3095a-aaa3-4c82-a333-7883f091f1c2', '_collection_name': 'PottyTraining'}, page_content='potty training boys and girls – potty genius                                             potty genius blog  games  shop ! potty genius blog — potty training boys — potty training girls — potty training methods  —  stories games  shop potty genius blog potty training boys and girls potty training is challenging regardless of your toddler’s gender. that said, potty training boys is a bit different than potty training girls. while it is obvious that males and females use the bathroom differently, there are some other distinct potty training differences parents may run into when potty training boys versus girls. by brittany tacket, ma brittany tackett is a 

In [80]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": itemgetter("query") | retriever, "query": itemgetter("query")} 
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm}
)

In [84]:
from pprint import pprint
answer = rag_chain.invoke(input={'query':"Should I use candy to help potty train my child?"})
pprint(answer)

{'response': 'Using candy to help potty train your child is a common practice, '
             "but it's not necessarily the most effective or recommended "
             'approach. Here are some things to consider:\n'
             '\n'
             '**Why candy might work:**\n'
             '\n'
             '1. **Temporary association**: Candy can associate the act of '
             'using the bathroom with something pleasant, like food.\n'
             '2. **Immediate satisfaction**: Receiving a treat can provide '
             'temporary relief and motivation for your child to use the '
             'potty.\n'
             '\n'
             '**However, there are also some potential drawbacks:**\n'
             '\n'
             '1. **Unintended consequences**: Over-reliance on candy might '
             'lead to dependence on treats as a means of getting your child to '
             'go pee or poo.\n'
             "2. **Lack of long-term habit formation**: Candy doesn't teach "
     